In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Tuple,List
from collections import Counter

In [3]:
file_name = "ld_clump_assoc.txt"
data = pd.read_csv(file_name,delimiter = "\t")

In [4]:
# creating nodes for cpgs and snps

cpgs:List[str] = data['CpG'].unique()
snps:List[str]= data['Top SNP'].unique()
lds:List[str] = data['LD clump'].unique() 
nodes_cpg = [{'id':cpg, 'group':1} for cpg in cpgs]
nodes_snp = [{'id':snp, 'group':2} for snp in snps]
nodes_LD = [{'id':ld, 'group':3} for ld in lds]
nodes = nodes_cpg + nodes_snp



In [5]:
# create cpg - snp link object  

cpg_snp_link_data = data.filter(items=['CpG','Top SNP'])
cpg_snp_link_data.columns = ['source', 'target']
cpg_snp_link_data['value'] = 1
cpg_snp_links = cpg_snp_link_data.to_dict(orient='records')

In [6]:
# create snp - snp link object by ld clump

snp_snp_link_data = data.filter(items=['Top SNP','LD clump'])
snp_snp_link_data.columns = ['source', 'target']
snp_snp_link_data['value'] = 1
snp_snp_links = snp_snp_link_data.to_dict(orient='records')

In [7]:
data

,CpG,Top SNP,LD clump,CpG chr,CpG pos,SNP chr,SNP pos,A1,A2,MAF,Beta,SE,P,FDR,N,n,Effects,Cis/Trans
0,cg06325811,1:753405_C_A,1:798400_A_G,1,796328,1,753405,C,A,0.1580,0.371869,0.049974,1.040000e-13,0.000000,2,1531,?+?+?,cis
1,cg16619049,1:798959_G_A,1:798400_A_G,1,805541,1,798959,G,A,0.2140,0.769962,0.061491,6.230000e-36,0.000000,2,1545,?++??,cis
2,cg13938959,1:838329_G_GC,1:851204_G_C,1,834183,1,838329,G,GC,0.1990,0.673932,0.045307,5.340000e-50,0.000000,2,1545,?++??,cis
3,cg13938959,1:930923_A_G,1:935459_A_G,1,834183,1,930923,A,G,0.1910,-0.395994,0.047221,5.300000e-17,0.000000,2,1545,?--??,cis
4,cg13938959,1:935222_C_A,1:1022037_C_T,1,834183,1,935222,C,A,0.4430,-0.204308,0.036335,1.920000e-08,0.000029,2,1545,?--??,cis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794329,cg23621893,20:60265147_A_G,20:60264314_T_C,22,50312169,20,60265147,A,G,0.1320,0.412361,0.047457,3.860000e-18,0.000000,2,1742,++???,trans
794330,cg23621893,20:60282962_C_T,20:60270858_A_G,22,50312169,20,60282962,C,T,0.4520,0.603907,0.032433,2.500000e-77,0.000000,3,1728,?+++?,trans
794331,cg15525648,1:2722848_C_T,1:2551660_T_G,22,50502484,1,2722848,C,T,0.4470,0.824447,0.068576,2.950000e-33,0.000000,3,1728,?+++?,trans
794332,cg09435190,17:92756_T_G,17:91617_CAAAG_C,22,50968297,17,92756,T,G,0.0546,-0.806188,0.081464,4.620000e-23,0.000000,2,1531,?-?-?,trans
